Import packages and set paths

In [1]:
import os
import sys
import emc2
import pint
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib as mpl
import pandas as pd
import numpy as np
import xarray as xr
import pyreadr
import netCDF4 as nc
import scipy
# import gini
from matplotlib.colors import LinearSegmentedColormap
ureg = pint.UnitRegistry()

# Navigate to Sonya's ACCESS-AM2 data directory
os.chdir('/g/data/jk72/slf563/ACCESS/output/cc407/hourly')
# Check current project directory
print("Project directory:", os.getcwd())

Project directory: /g/data/jk72/slf563/ACCESS/output/cc407/hourly


In [2]:
# Load the model data

# Set the model output path
model_output = 'cc407a.ph.met20180219.nc'

# Load data
met20180219 = xr.open_dataset(model_output)

# # Load the old model data for comparison
# model_output_old = 'old/cc407a.ph.met20180219.nc'
# met20180219_old = xr.open_dataset(model_output_old)
# # Print the new variables
# print("New variables in met20180219 compared to met20180219_old:")
# for variable in set(met20180219.variables) - set(met20180219_old.variables):
#     print(variable)

In [3]:
# met20180219

In [4]:
# Load MARCUS coordinate data

# Load MARCUS coordinate data and reformat as datetime62
marcus_coordinates_v1_3 = pd.read_csv('/g/data/jk72/ck4840/projects/emc2/data/marcus_coordinates/marcus_coordinates_v1_3.csv')
marcus_coordinates_v1_3['datetime_day'] = pd.to_datetime(marcus_coordinates_v1_3['datetime_day'])

# Extract the date from the 'time' coordinate in the model data and filter the MARCUS coordinate data to the model data date
marcus_coordinates = marcus_coordinates_v1_3[marcus_coordinates_v1_3['datetime_day'].dt.date == pd.to_datetime(met20180219['time'].values).date[0]]

# Convert MARCUS coordinate data to XArray dataset and convert datetime_hour variable to a coordinate
marcus_coordinates = xr.Dataset.from_dataframe(marcus_coordinates)
marcus_coordinates = marcus_coordinates.set_coords(['datetime_hour'])

In [5]:
# Subset the met20180219 model data to the Underway coordinates

# Ensure that the time coordinates are the same data type
marcus_coordinates['datetime_hour'] = marcus_coordinates['datetime_hour'].astype('datetime64[ns]')
met20180219['time'] = met20180219['time'].astype('datetime64[ns]')

# Extract the time values from the ACCESS model data
time_values = met20180219['time'].values

# Initalise list to store the individual datasets from each iteration
list_of_datasets = []

# Loop over each time step in the ship dataset
for i in range(len(marcus_coordinates['datetime_hour'])):
    # Get the lat, lon, and time values for each step
    lat = marcus_coordinates['latitude'].isel(index=i)
    lon = marcus_coordinates['longitude'].isel(index=i)
    time = marcus_coordinates['datetime_hour'].isel(index=i)
    
    # Interpolating the entire dataset at the ship's position
    met20180219_interpolated = met20180219.interp(
        lat=lat.item(), 
        lon=lon.item(), 
        time=time_values[i], 
        method='nearest'
    )
    
    # Append each interpolated dataset to the list
    list_of_datasets.append(met20180219_interpolated)

# Concatenate all datasets along a new dimension 'index'
met20180219_subset = xr.concat(list_of_datasets, dim='index')
# Now, the combined_dataset contains the interpolated values for each time step in a single dataset

# Add a zero array to the data

# Create a DataArray filled with zeros and specify dimensions
zeros_data = xr.DataArray(
    data=0.0,  # Filling with zeros
    dims=['time1', 'z1_hybrid_height'],  # Dimensions you want
    coords={'time1': met20180219_subset['time1'], 'z1_hybrid_height': met20180219_subset['z1_hybrid_height']}
)

# Add this DataArray to the dataset
met20180219_subset['zeros_var'] = zeros_data

# Add a 2D z_values variable to the dataset
z1_values = met20180219_subset['z1_hybrid_height'].values
z_values = z1_values[:, np.newaxis] * np.ones(len(met20180219_subset['time1']))
met20180219_subset['z'] = (('z1_hybrid_height', 'time1'), z_values)
met20180219_subset['z'].attrs['units'] = 'meters'
met20180219_subset['z'] = met20180219_subset['z'].transpose('time1', 'z1_hybrid_height')

print(met20180219_subset)

<xarray.Dataset>
Dimensions:           (lat_v: 50, time1: 24, z0_hybrid_height: 85,
                       z1_hybrid_height: 85, index: 24)
Coordinates:
  * lat_v             (lat_v) float64 -90.0 -88.75 -87.5 ... -31.25 -30.0 -28.75
  * time1             (time1) datetime64[ns] 2018-02-19T00:20:00.000000256 .....
  * z0_hybrid_height  (z0_hybrid_height) float64 10.0 36.67 ... 8.205e+04
  * z1_hybrid_height  (z1_hybrid_height) float64 20.0 53.33 ... 7.91e+04 8.5e+04
    lat               (index) float64 -67.86 -67.95 -68.04 ... -68.57 -68.57
    lon               (index) float64 73.48 73.99 74.52 ... 77.93 77.93 77.93
    time              (index) datetime64[ns] 2018-02-19T01:00:00 ... 2018-02-20
Dimensions without coordinates: index
Data variables: (12/82)
    va                (index, z0_hybrid_height, lat_v) float32 -3.629 ... 10.81
    theta             (index, z1_hybrid_height) float32 268.5 268.5 ... 7.22e+03
    hus               (index, z1_hybrid_height) float32 0.001917 ... 2.4

In [6]:
# Estimate effective radius and add it to the dataset

met20180219_subset['lat'] = xr.DataArray(data=met20180219_subset['lat'], dims=('index'))

# # field4210 is cloud droplet number concentration
# # field254, field12... is mass mixing ratio, field4210/field254... is number mixing ratio
 
rcl = ((((met20180219_subset.field254/(met20180219_subset.field4210/met20180219_subset.field254))/1000)*3)/(4*np.pi))**(1./3)
rci = ((((met20180219_subset.field12/(met20180219_subset.field4210/met20180219_subset.field12))/500)*3)/(4*np.pi))**(1./3)
rpl = ((((met20180219_subset.field394/(met20180219_subset.field4210/met20180219_subset.field394))/1000)*3)/(4*np.pi))**(1./3)
rpi = ((((met20180219_subset.field396/(met20180219_subset.field4210/met20180219_subset.field396))/250)*3)/(4*np.pi))**(1./3)

# rcl

# re_mccl_da = xr.DataArray(data=rcl, dims=('time1', 'z1_hybrid_height'), coords={'lat': met20180219_subset['lat']})
# met20180219_subset['re_mccl'] = re_mccl_da

# met20180219_subset['re_mccl'] = xr.DataArray(data=rcl,dims=('time1','z1_hybrid_height'))
# met20180219_subset['re_mcci'] = xr.DataArray(data=rci,dims=('time1','z1_hybrid_height'))
# met20180219_subset['re_mcpi'] = xr.DataArray(data=rpi,dims=('time1','z1_hybrid_height'))
# met20180219_subset['re_mcpl'] = xr.DataArray(data=rpl,dims=('time1','z1_hybrid_height'))

met20180219_subset

<xarray.Dataset>
Dimensions:           (lat_v: 50, time1: 24, z0_hybrid_height: 85,
                       z1_hybrid_height: 85, index: 24)
Coordinates:
  * lat_v             (lat_v) float64 -90.0 -88.75 -87.5 ... -31.25 -30.0 -28.75
  * time1             (time1) datetime64[ns] 2018-02-19T00:20:00.000000256 .....
  * z0_hybrid_height  (z0_hybrid_height) float64 10.0 36.67 ... 8.205e+04
  * z1_hybrid_height  (z1_hybrid_height) float64 20.0 53.33 ... 7.91e+04 8.5e+04
    lat               (index) float64 -67.86 -67.95 -68.04 ... -68.57 -68.57
    lon               (index) float64 73.48 73.99 74.52 ... 77.93 77.93 77.93
    time              (index) datetime64[ns] 2018-02-19T01:00:00 ... 2018-02-20
Dimensions without coordinates: index
Data variables: (12/82)
    va                (index, z0_hybrid_height, lat_v) float32 -3.629 ... 10.81
    theta             (index, z1_hybrid_height) float32 268.5 268.5 ... 7.22e+03
    hus               (index, z1_hybrid_height) float32 0.001917 ... 2.431e-07
    field12           (index, z1_hybrid_height) float32 2.546e-05 ... 0.0
    bldepth           (index) float32 822.2 822.2 822.2 ... 1.562e+03 718.2
    sic               (index) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                ...
    lwp               (index) float32 0.02846 0.03968 0.0438 ... 0.08286 0.01431
    clivi             (index) float32 0.0466 0.04961 0.04771 ... 0.03414 0.03854
    field30453        (index) float32 8.397e+03 8.397e+03 ... 8.251e+03
    field30461        (index) float32 4.304 4.257 4.194 ... 3.653 3.592 3.616
    zeros_var         (time1, z1_hybrid_height) float64 0.0 0.0 0.0 ... 0.0 0.0
    z                 (time1, z1_hybrid_height) float64 20.0 53.33 ... 8.5e+04
Attributes:
    history:  /scratch/jk72/slf563/cylc-run/u-cc407/share/data/History_Data/c...

In [48]:
# Get the 'field4210' (ncl) and 'field254' (qcl) variables as NumPy arrays
ncl = met20180219_subset['field4210'].values
qcl = met20180219_subset['field254'].values

# Find the first index where both ncl and qcl are nonzero
index = np.where((ncl > 0) & (qcl > 0))[0][0]

# Extract the values at the found index, and select only subindex 6
ncl_value = ncl[index]#[6:11]
qcl_value = qcl[index]#[6:11]

# Print the values
print("Index:", index)
print("ncl (field4210) value at the index:", ncl_value)
print("qcl (field254) value at the index:", qcl_value)

Index: 0
ncl (field4210) value at the index: [       0.        0.        0.        0.        0.        0. 50331648.
 50331648. 50331648. 50331648. 50331648. 50331648.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.        0.        0.        0.        0.        0.        0.
        0.]
qcl (field254) value at the index: [0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00

In [ ]:
# Extract cloud droplet number concentration and mass mixing ratios
ncl = met20180219_subset['field4210'].values
qcl = met20180219_subset['field254'].values
qpl = met20180219_subset['field394'].values
qci = met20180219_subset['field12'].values
qpi = met20180219_subset['field396'].values
qpg = met20180219_subset['field395'].values

# These variables, like their parent dataset, have a shape of (24, 85)

# Find the first index where both ncl and qcl are nonzero
# index = np.where((ncl > 0) & (qcl > 0))
indices = np.where(ncl > 0)

# Extract the values at the found index, and select only subindex 6
ncl_value = ncl[index]
qcl_value = qcl[index]

# Print the values
print("Index:", index)
print("ncl (field4210) value at the index:", ncl_value)
print("qcl (field254) value at the index:", qcl_value)

In [42]:
# re = ((3*qc)^(1/3))/((4*1000*math.pi*nc)^(1/3))
# Therefore, for extreme re values, couple the maximum qc with the minimum nc

import math

re = ((3*qcl_value)**(1/3))/((4*1000*math.pi*ncl_value)**(1/3))
print(re)

[2.7781030e-06 4.5521670e-06 6.2340987e-06 7.4315817e-06 2.6736668e-06]


In [7]:
# Examine required variables

cl = met20180219_subset['field254'].astype('float64')
ci = met20180219['field12'].astype('float64')

p_3d = met20180219_subset['field408'].astype('float64') # self.p_field = "p_3d"
z = met20180219_subset['z1_hybrid_height'] # self.z_field = "z"
t = met20180219_subset['ta'].astype('float64') # self.T_field = "t"
plm = 'field408' # self.height_dim = "plm"
time = 'time' # self.time_dim = "time"

print("cl dimensions (time, height, lat, lon):", cl.shape, "cl data type:", cl.dtype)
print("ci dimensions (time, height, lat, lon):", ci.shape, "ci data type", ci.dtype)
print("p_3d dimensions (time, height, lat, lon):", p_3d.shape, "p_3d data type", p_3d.dtype)
print("z dimension (height):", z.shape, "z data type", z.dtype)
print("t dimensions (time, height, lat, lon):", t.shape, "t data type", t.dtype)

time_dim = 24
height_dim = 85
lat_dim = 1
lon_dim = 1

# # Create zero arrays with the specified dimensions
# zero_array = np.zeros((time_dim, height_dim, lat_dim, lon_dim))

# print("zero_array dimensions (time, height, lat, lon):", zero_array.shape, "zero_array data type", zero_array.dtype)

cl dimensions (time, height, lat, lon): (24, 85) cl data type: float64
ci dimensions (time, height, lat, lon): (24, 85, 49, 71) ci data type float64
p_3d dimensions (time, height, lat, lon): (24, 85) p_3d data type float64
z dimension (height): (85,) z data type float64
t dimensions (time, height, lat, lon): (24, 85) t data type float64


In [14]:
class ACCESS_AM2(emc2.core.Model):
    def __init__(self, file_path):
       """
       This loads an ACCESS-AM2 simulation with all of the necessary parameters for EMC^2 to run.
       
       Parameters
       ----------
       file_path: str
           Path to an ACCESS-AM2 simulation.
       time_range: tuple, list, or array, typically in datetime64 format
           Two-element array with starting and ending of time range.
       load_processed: bool
           If True, treating the 'file_path' variable as an EMC2-processed dataset; thus skipping dimension stacking as part of pre-processing.
       """
       
       super().__init__()
       
       # Bulk density
       self.Rho_hyd = {'cl': 1000. * ureg.kg / (ureg.m**3),'ci': 500. * ureg.kg / (ureg.m**3),
                       'pl': 1000. * ureg.kg / (ureg.m**3),'pi': 250. * ureg.kg / (ureg.m**3)}
       self.fluffy = {'ci': 0.5 * ureg.dimensionless, 'pi': 0.5 * ureg.dimensionless}
       # Lidar ratio
       self.lidar_ratio = {'cl': 18. * ureg.dimensionless,
                           'ci': 24. * ureg.dimensionless,
                           'pl': 5.5 * ureg.dimensionless,
                           'pi': 24.0 * ureg.dimensionless}
       # Lidar LDR per hydrometeor mass content
       self.LDR_per_hyd = {'cl': 0.03 * 1 / (ureg.kg / (ureg.m**3)),
                           'ci': 0.35 * 1 / (ureg.kg / (ureg.m**3)),
                           'pl': 0.1 * 1 / (ureg.kg / (ureg.m**3)),
                           'pi': 0.40 * 1 / (ureg.kg / (ureg.m**3))}
       # a, b in V = aD^b
       self.vel_param_a = {'cl': 3e-7, 'ci': 700., 'pl': 841.997, 'pi': 11.72}
       self.vel_param_b = {'cl': 2. * ureg.dimensionless,
                           'ci': 1. * ureg.dimensionless,
                           'pl': 0.8 * ureg.dimensionless,
                           'pi': 0.41 * ureg.dimensionless}
       super()._add_vel_units()
       # Names of mixing ratios of species
       # What is the difference between field254 and field392? Also, what is the difference between field12 and field393?
       self.q_names = {'cl': 'field254', 'ci': 'field12', 'pl': 'field394', 'pi': 'field396'}
       # Number concentration of each species
       self.N_field = {'cl': 'field4210', 'ci': 'field4210', 'pl': 'field4210', 'pi': 'field4210'} # Need number concentrations of each species
       # Convective fraction
       self.conv_frac_names = {'cl': 'zeros_var', 'ci': 'zeros_var', 'pl': 'zeros_var', 'pi': 'zeros_var'} # Closest matches are 2D CONVECTIVE RAINFALL RATE and CONVECTIVE SNOWFALL RATE
       self.conv_frac_names_for_rad = {'cl': 'zeros_var', 'ci': 'zeros_var', 'pl': 'zeros_var', 'pi': 'zeros_var'}
       # Stratiform fraction
       self.strat_frac_names = {'cl': 'zeros_var', 'ci': 'zeros_var', 'pl': 'zeros_var', 'pi': 'zeros_var'} # Closest match may be 2D LARGE SCALE RAINFALL RATE and LARGE SCALE SNOWFALL RATE
       self.strat_frac_names_for_rad = {'cl': 'zeros_var', 'ci': 'zeros_var', 'pl': 'zeros_var', 'pi': 'zeros_var'}
       # Effective radius
       self.re_fields = {'cl': 'zeros_var', 'ci': 'zeros_var', 'pi': 'zeros_var', 'pl': 'zeros_var'} # Need effective radius of each species
       # Convective mixing ratio
       self.q_names_convective = {'cl': 'zeros_var', 'ci': 'zeros_var', 'pl': 'zeros_var', 'pi': 'zeros_var'}
       # Stratiform mixing ratio
       self.q_names_stratiform = {'cl': 'zeros_var', 'ci': 'zeros_var', 'pl': 'zeros_var', 'pi': 'zeros_var'}
       # Water vapor mixing ratio
       self.q_field = "hus"
       # Pressure
       self.p_field = "field408"
       # Height
       self.z_field = "z"
       # Temperature
       self.T_field = "ta"
       # Name of height dimension
       self.height_dim = "z1_hybrid_height"
       # Name of time dimension
       self.time_dim = "time1"
       self.hyd_types = ["cl", "ci", "pl", "pi"]
       self.process_conv = False
       self.model_name = "ACCESS_AM2"
       # self.ds = xr.open_dataset(file_path)
       self.ds = met20180219_subset

In [15]:
# met20180219 = xr.open_dataset(model_output)
my_model = ACCESS_AM2(met20180219_subset)
print(dir(my_model))
# my_model.re_fields

['LDR_per_hyd', 'N_field', 'Rho_hyd', 'T_field', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_vel_units', '_crop_bounding_box', '_crop_time_range', '_prepare_variables', 'asp_ratio_func', 'check_and_stack_time_lat_lon', 'consts', 'conv_frac_names', 'conv_frac_names_for_rad', 'conv_re_fields', 'ds', 'finalize_subcol_fields', 'fluffy', 'height_dim', 'hyd_types', 'hydrometeor_classes', 'ice_hyd_types', 'lambda_field', 'lat_dim', 'lidar_ratio', 'load_subcolumns_from_netcdf', 'lon_dim', 'mcphys_scheme', 'model_name', 'mu_field', 'num_hydrometeor_classes', 'num_subcolumns', 'p_field', 'permute_dims_for_processing', 'process_conv', 'q_field', 'q_names', 'q_names_convective', 'q

In [16]:
from emc2.core.instruments import HSRL

# Create an instance of the HSRL instrument
MPL = HSRL()
print(dir(MPL))

['K_w', 'OD_from_sfc', 'R_d', 'Z_min_1km', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'beta_p_phase_thresh', 'bulk_table', 'c', 'ds', 'eps_liq', 'eta', 'ext_OD', 'freq', 'g', 'gain', 'instrument_class', 'instrument_str', 'location_code', 'lr', 'mie_table', 'pr_noise_ge', 'pr_noise_md', 'pt', 'read_arm_netcdf_file', 'rho_i', 'rho_l', 'scat_table', 'scatterer', 'tau_ge', 'tau_md', 'theta', 'wavelength']


In [17]:
# Create simulated data
marcus_mpl_access_model = emc2.simulator.main.make_simulated_data(my_model, MPL, N_columns = 1, do_classify = True, convert_zeros_to_nan = True, skip_subcol_gen = False, use_rad_logic = True)

## Creating subcolumns...
No convective processing for ACCESS_AM2
num_subcolumns == 1 (subcolumn generator turned off); setting subcolumns frac fields to 1 for startiform cl and ci based on q > 0. kg/kg


ValueError: cannot reindex or align along dimension 'subcolumn' because of conflicting dimension sizes: {1, 4} (note: an index is found along that dimension with size=4)

In [ ]:
# Save simulated data
import pickle

with open('/g/data/jk72/ck4840/projects/emc2/data/access/marcus_mpl_access_model.pkl', 'wb') as file:
    pickle.dump(marcus_mpl_access_model, file)

In [12]:
# Load simulated data
import pickle

with open('/g/data/jk72/ck4840/projects/emc2/data/access/marcus_mpl_access_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

In [23]:
print(dir(marcus_mpl_access_model))
# print(type(marcus_mpl_access_model))

['LDR_per_hyd', 'N_field', 'Rho_hyd', 'T_field', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_add_vel_units', '_crop_bounding_box', '_crop_time_range', '_prepare_variables', 'asp_ratio_func', 'check_and_stack_time_lat_lon', 'consts', 'conv_frac_names', 'conv_frac_names_for_rad', 'conv_re_fields', 'ds', 'finalize_subcol_fields', 'fluffy', 'height_dim', 'hyd_types', 'hydrometeor_classes', 'ice_hyd_types', 'lambda_field', 'lat_dim', 'lidar_ratio', 'load_subcolumns_from_netcdf', 'lon_dim', 'mcphys_scheme', 'model_name', 'mu_field', 'num_hydrometeor_classes', 'num_subcolumns', 'p_field', 'permute_dims_for_processing', 'process_conv', 'q_field', 'q_names', 'q_name

In [13]:
print(dir(my_model))

['LDR_per_hyd', 'N_field', 'Rho_hyd', 'T_field', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_vel_units', '_crop_bounding_box', '_crop_time_range', '_prepare_variables', 'asp_ratio_func', 'check_and_stack_time_lat_lon', 'consts', 'conv_frac_names', 'conv_frac_names_for_rad', 'conv_re_fields', 'ds', 'finalize_subcol_fields', 'fluffy', 'height_dim', 'hyd_types', 'hydrometeor_classes', 'ice_hyd_types', 'lambda_field', 'lat_dim', 'lidar_ratio', 'load_subcolumns_from_netcdf', 'lon_dim', 'mcphys_scheme', 'model_name', 'mu_field', 'num_hydrometeor_classes', 'num_subcolumns', 'p_field', 'permute_dims_for_processing', 'process_conv', 'q_field', 'q_names', 'q_names_convective', 'q

In [ ]:
# ls_rainfall = met20180219.variables['field4203']
# conv_rainfall = met20180219.variables['field5205']
# total_rainfall = met20180219.variables['prrn']
# ls_snowfall = met20180219.variables['field4204']
# conv_snowfall = met20180219.variables['field5206']
# total_snowfall = met20180219.variables['prsn']
# total_precipitation = met20180219.variables['pr']
# mask = (ls_rainfall > 0) & (conv_rainfall > 0) & (total_rainfall > 0) & \
#        (ls_snowfall > 0) & (conv_snowfall > 0) & (total_snowfall > 0) & \
#        (total_precipitation > 0)
# time_indices, lat_indices, lon_indices = np.where(mask)
# print(time_indices)
# print(lat_indices)
# print(lon_indices)
# time_index = 0
# lat_index = 21
# lon_index = 9
# print("Time: 0")
# print("Latitude: 21")
# print("Longitude: 9")
# print("LARGE SCALE RAINFALL RATE: ", ls_rainfall[time_index, lat_index, lon_index].values, "KG/M2/S")
# print("CONVECTIVE RAINFALL RATE: ", conv_rainfall[time_index, lat_index, lon_index].values, "KG/M2/S")
# print("TOTAL RAINFALL RATE: LS+CONV: ", total_rainfall[time_index, lat_index, lon_index].values, "KG/M2/S")
# print("LARGE SCALE SNOWFALL RATE: ", ls_snowfall[time_index, lat_index, lon_index].values, "KG/M2/S")
# print("CONVECTIVE SNOWFALL RATE: ", conv_snowfall[time_index, lat_index, lon_index].values, "KG/M2/S")
# print("TOTAL SNOWFALL RATE: LS+CONV: ", total_snowfall[time_index, lat_index, lon_index].values, "KG/M2/S")
# print("TOTAL PRECIPITATION RATE: ", total_precipitation[time_index, lat_index, lon_index].values, "KG/M2/S")

In [107]:
# class ModelE(Model):
#     def __init__(self, file_path, time_range=None, load_processed=False):
#         """
#         This loads a ModelE simulation with all of the necessary parameters for EMC^2 to run.

#         Parameters
#         ----------
#         file_path: str
#             Path to a ModelE simulation.
#         time_range: tuple, list, or array, typically in datetime64 format
#             Two-element array with starting and ending of time range.
#         load_processed: bool
#             If True, treating the 'file_path' variable as an EMC2-processed dataset; thus skipping
#             dimension stacking as part of pre-processing.
#         """
        
#         super().__init__()
        
#         self.Rho_hyd = {'cl': 1000. * ureg.kg / (ureg.m**3), 'ci': 500. * ureg.kg / (ureg.m**3),
#                         'pl': 1000. * ureg.kg / (ureg.m**3), 'pi': 250. * ureg.kg / (ureg.m**3)}
#         self.fluffy = {'ci': 0.5 * ureg.dimensionless, 'pi': 0.5 * ureg.dimensionless}
#         self.lidar_ratio = {'cl': 18. * ureg.dimensionless,
#                             'ci': 24. * ureg.dimensionless,
#                             'pl': 5.5 * ureg.dimensionless,
#                             'pi': 24.0 * ureg.dimensionless}
#         self.LDR_per_hyd = {'cl': 0.03 * 1 / (ureg.kg / (ureg.m**3)),
#                             'ci': 0.35 * 1 / (ureg.kg / (ureg.m**3)),
#                             'pl': 0.1 * 1 / (ureg.kg / (ureg.m**3)),
#                             'pi': 0.40 * 1 / (ureg.kg / (ureg.m**3))}
#         self.vel_param_a = {'cl': 3e7, 'ci': 700., 'pl': 841.997, 'pi': 11.72}
#         self.vel_param_b = {'cl': 2. * ureg.dimensionless,
#                             'ci': 1. * ureg.dimensionless,
#                             'pl': 0.8 * ureg.dimensionless,
#                             'pi': 0.41 * ureg.dimensionless}
#         super()._add_vel_units()
#         self.q_field = "q"
#         self.N_field = {'cl': 'ncl', 'ci': 'nci', 'pl': 'npl', 'pi': 'npi'}
#         self.p_field = "p_3d"
#         self.z_field = "z"
#         self.T_field = "t"
#         self.height_dim = "p"
#         self.time_dim = "time"
#         self.conv_frac_names = {'cl': 'cldmccl', 'ci': 'cldmcci', 'pl': 'cldmcpl', 'pi': 'cldmcpi'}
#         self.strat_frac_names = {'cl': 'cldsscl', 'ci': 'cldssci', 'pl': 'cldsspl', 'pi': 'cldsspi'}
#         self.conv_frac_names_for_rad = {'cl': 'cldmcr', 'ci': 'cldmcr',
#                                         'pl': 'cldmcpl', 'pi': 'cldmcpi'}
#         self.strat_frac_names_for_rad = {'cl': 'cldssr', 'ci': 'cldssr',
#                                          'pl': 'cldssr', 'pi': 'cldssr'}
#         self.conv_re_fields = {'cl': 're_mccl', 'ci': 're_mcci', 'pi': 're_mcpi', 'pl': 're_mcpl'}
#         self.strat_re_fields = {'cl': 're_sscl', 'ci': 're_ssci', 'pi': 're_sspi', 'pl': 're_sspl'}
#         self.q_names_convective = {'cl': 'QCLmc', 'ci': 'QCImc', 'pl': 'QPLmc', 'pi': 'QPImc'}
#         self.q_names_stratiform = {'cl': 'qcl', 'ci': 'qci', 'pl': 'qpl', 'pi': 'qpi'}
#         self.hyd_types = ["cl", "ci", "pl", "pi"]

#         if load_processed:
#             self.ds = xr.Dataset()
#             self.load_subcolumns_from_netcdf(file_path)
#         else:
#             self.ds = read_netcdf(file_path)
#         if np.logical_and("level" in self.ds.coords, not "p" in self.ds.coords):
#             self.height_dim = "level"

#         # crop specific model output time range (if requested)
#         if time_range is not None:
#             if np.issubdtype(time_range.dtype, np.datetime64):
#                 super()._crop_time_range(time_range)
#             else:
#                 raise RuntimeError("input time range is not in the required datetime64 data type")

#         if not load_processed:

#             # stack dimensions in the case of a regional output or squeeze lat/lon dims if exist and len==1
#             super().check_and_stack_time_lat_lon(file_path=file_path)

#             # ModelE has pressure units in mb, but pint only supports hPa
#             self.ds["p_3d"].attrs["units"] = "hPa"

#         self.model_name = "ModelE3"

In [108]:
# class MPL(emc2.core.Instrument):
#     def __init__(self):
#         """
#         This stores the information for the Micropulse Lidar.
#         """
#         super().__init__(wavelength=0.532 * ureg.micrometer)
#         self.instrument_class = "lidar"
#         self.instrument_str = "MPL"
#         self.ext_OD = 4
#         self.K_w = np.nan
#         self.eps_liq = (1.337273 + 1.7570744e-9j)**2
#         self.pt = np.nan
#         self.theta = np.nan
#         self.gain = np.nan
#         self.Z_min_1km = np.nan
#         self.lr = np.nan
#         self.pr_noise_ge = np.nan
#         self.pr_noise_md = np.nan
#         self.tau_ge = np.nan
#         self.tau_md = np.nan
#         self.OD_from_sfc = True